# [Car Evaluation Dataset](https://archive.ics.uci.edu/dataset/19/car+evaluation)

Car Evaluation Database was derived from a simple hierarchical decision model originally developed for the demonstration of DEX, M. Bohanec, V. Rajkovic: Expert system for decision making. Sistemica 1(1), pp. 145-157, 1990.

### Estrutura do dataset

| Alvo  | Valores |
| ------------- | ------------- |
|class| {unacc, acc, good, vgood}|

| Features  | Valores |
| ------------- | ------------- |
|buying| {vhigh, high, med, low}|
|maint| {vhigh, high, med, low}|
|doors| {2, 3, 4, 5more}|
|persons| {2, 4, more}|
|lug_boot| {small, med, big}|
|safety| {low, med, high}|

### Instalação das dependências

In [12]:
# !pip install ucimlrepo
# !pip install tensorflow
# !pip install sklearn
# !pip install pandas

In [13]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from ucimlrepo import fetch_ucirepo 
pd.options.mode.copy_on_write = True

### Importando o dataset

In [14]:
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
Y = car_evaluation.data.targets 

# Transform labels to int
labels = Y["class"].unique()
for i in range(len(labels)):
  Y.loc[Y['class']==labels[i], 'class'] = i


### One-hot Encoding das features

In [15]:
X = pd.get_dummies(X, dtype=int)

### Dividindo os conjuntos de teste e treino

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size= 0.3, random_state = 28)

### One-Hot Enconding dos Targets

In [17]:
y_train = to_categorical(y_train) 
y_true = list(y_test['class'])
y_test = to_categorical(y_test) 

### Perceptron

In [18]:
model = tf.keras.Sequential([
    Flatten(input_shape=(len(X.columns), )),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(y_train[0]), activation='softmax')
])

In [19]:
model.compile(
  loss='categorical_crossentropy', 
  optimizer='sgd', 
  metrics=['Accuracy', 'Precision', 'Recall', 'F1Score']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 21)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               2816      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 11332 (44.27 KB)
Trainable params: 11332 (44.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Treinamento

In [20]:
model.fit(x_train, y_train, epochs=200, batch_size=32, verbose=1)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/200


38/38 [==============================] - 1s 919us/step - loss: 1.2195 - Accuracy: 0.5261 - precision: 1.0000 - recall: 0.0017 - f1_score: 0.2612   
Epoch 2/200
38/38 [==============================] - 0s 946us/step - loss: 0.9373 - Accuracy: 0.6923 - precision: 0.8499 - recall: 0.3606 - f1_score: 0.2209
Epoch 3/200
38/38 [==============================] - 0s 865us/step - loss: 0.8165 - Accuracy: 0.6882 - precision: 0.7590 - recall: 0.6592 - f1_score: 0.2038
Epoch 4/200
38/38 [==============================] - 0s 1ms/step - loss: 0.7578 - Accuracy: 0.6882 - precision: 0.7233 - recall: 0.6832 - f1_score: 0.2038
Epoch 5/200
38/38 [==============================] - 0s 892us/step - loss: 0.7172 - Accuracy: 0.6890 - precision: 0.7292 - recall: 0.6816 - f1_score: 0.2057
Epoch 6/200
38/38 [==============================] - 0s 892us/step - loss: 0.6825 - Accuracy: 0.6981 - precision: 0.7569 - recall: 0.6799 - f1_score: 0.2271
Epoch 7/200
38/38 [==============================] - 0s 865us/step - 

[0.07858027517795563,
 0.9807322025299072,
 0.9824561476707458,
 0.9710982441902161,
 array([0.99202126, 0.9565218 , 0.9756098 , 0.8947368 ], dtype=float32)]

In [21]:
predictions = model.predict(x_test)
predictions = [list(p).index(max(p)) for p in predictions]


17/17 [==============================] - 0s 625us/step


In [22]:
print(f'Acurácia obtida: {accuracy_score(y_true, predictions) * 100:.2f}%')
print(f'Precisão obtida: {precision_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'Recall obtido: {recall_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'F1 Score obtida: {f1_score(y_true, predictions, average="weighted") * 100:.2f}%')


Acurácia obtida: 98.07%
Precisão obtida: 98.15%
Recall obtido: 98.07%
F1 Score obtida: 98.08%
